# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

AttributeError: module 'cftime' has no attribute 'datetime'

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
plot = city_data_df.hvplot.scatter(
    x="Lng",  
    y="Lat",  
    size="Humidity",  
    xlim=(-180, 180),  
    ylim=(-90, 90),   
    color = "blue",  
    tiles = "random title", 
    xlabel="x", 
    ylabel="y",   
    hover_cols=["City", "Humidity"],  
)

# Display the map
plot

ImportError: Geographic projection support requires: cartopy, geoviews.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
selected_cities_df = city_data_df[
    (city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) &  
    (city_data_df["Wind Speed"] < 4.5) &                                  
    (city_data_df["Cloudiness"] == 0)                                    
]

# Drop any rows with null values
selected_cities_df = selected_cities_df.dropna()

# Display sample data
selected_cities_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
83,83,astrakhan,46.3497,48.0408,21.93,73,0,1.00,RU,1690314189
92,92,nanpiao,41.0982,120.7479,21.29,93,0,1.42,CN,1690313927
121,121,zharkent,44.1628,80.0000,24.50,22,0,1.84,KZ,1690314197
143,143,permet,40.2336,20.3517,26.97,34,0,2.83,AL,1690314202
171,171,yalta,44.5000,34.1667,22.97,67,0,0.24,UA,1690314208
254,254,devils lake,48.1128,-98.8651,26.99,78,0,3.60,US,1690314230
342,342,dicle,38.3751,40.0722,23.61,40,0,3.43,TR,1690314253
365,365,shingu,33.7333,135.9833,24.29,90,0,1.83,JP,1690314263
403,403,dryden,49.7833,-92.7503,26.16,53,0,3.09,CA,1690314271
424,424,kaspiyskiy,45.3929,47.3707,24.85,66,0,1.46,RU,1690314276


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = selected_cities_df.copy()

columns_to_drop = ["City_ID", "Max Temp", "Cloudiness", "Wind Speed", "Date"]
hotel_df.drop(columns=columns_to_drop, inplace=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.assign(Hotel_Name="")
desired_columns = ["City", "Country", "Lat", "Lng", "Humidity","Hotel Name"]
hotel_df = hotel_df.reindex(columns=desired_columns)

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
83,astrakhan,RU,46.3497,48.0408,73,NaN
92,nanpiao,CN,41.0982,120.7479,93,NaN
121,zharkent,KZ,44.1628,80.0000,22,NaN
143,permet,AL,40.2336,20.3517,34,NaN
171,yalta,UA,44.5000,34.1667,67,NaN
254,devils lake,US,48.1128,-98.8651,78,NaN
342,dicle,TR,38.3751,40.0722,40,NaN
365,shingu,JP,33.7333,135.9833,90,NaN
403,dryden,CA,49.7833,-92.7503,53,NaN
424,kaspiyskiy,RU,45.3929,47.3707,66,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [4]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "type": "accommodation",
    "limit": 1
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"distance(lat,lon,{latitude},{longitude}) lt {radius}"
    params["bias"] = f"proximity({latitude},{longitude})"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

NameError: name 'geoapify_key' is not defined

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE